In [1]:
#| default_exp regress_ml

# Train ML model

In [2]:
#|hide
from nbdev.showdoc import *

In [29]:
#|hide
#|export
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split


In [55]:
#|export

class TrainRegression():
       
       def __init__(self,
               df_path : str , # path to dataframe to be used to train. File should be CSV file
               ground_truth_col: str, # name of the column with true data to train
               test_size : float , #size of data to be used for test 

               columns_to_remove : list[str]=None , #columns not to use for trainning the model. These columns will be removed.
               ):
             self.df = pd.read_csv(df_path)
             self.columns_to_remove = columns_to_remove
             self.ground_truth_col = ground_truth_col

             self.test =self._load_df_split_data(self)
             self.test_size = test_size



       def _load_df_split_data(self):
               #load dataframe
               if self.columns_to_remove!= None:
                     self.df = self.df.drop(self.columns_to_remove,axis=1)
          
               # split to x,y and train test data
               self.x = self.df.drop(self.ground_truth_col,axis=1)
               self.y = self.df[self.ground_truth_col].values

               #split data to train and test
               self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
                      self.x, self.y, test_size=self.test_size, random_state=42)

               return self.x_train, self.x_test, self.y_train, self.y_test



In [56]:
instance = TrainRegression(
  df_path=r"D:\git\hyperspectral_tools\nbs\data\resampled\resampled_sen2.csv",
  ground_truth_col = "TOC",
  test_size = 0.25,
  columns_to_remove = ['Unnamed: 0.1', 'Unnamed: 0', 'Lon', 'Lat', 'clay', 'silt','sand', 'NI']    
                    )

instance._load_df_split_data()


TypeError: TrainRegression._load_df_split_data() takes 1 positional argument but 2 were given